<a href="https://colab.research.google.com/github/maipatana/tuar686/blob/master/AR686_Day2_Supervised_and_Unsupervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intoduction

์Notebook นี้เป็นส่วนหนึ่งของวิชา AR686 คณะสถาปัตยกรรมศาสตร์และการผังเมือง มหาวิทยาลัยธรรมศาสตร์

Notebook นี้เป็น tutorial ในเรื่องของ Supervised และ Unsupervised Learning เพื่อวิเคราะห์ข้อมูลรูปแบบต่างๆ

## Recommendations

เรามาลองสร้าง Recommendations บนเว็บไซค์ของเราให้กับผู้ที่เข้ามาใช้งาน ตามข้อมูลที่ผู้ใช้งานเคยกดให้ดาวผลงานต่างๆบนเว็บไซต์แสดงผลงานของบริษัทเรา (ดาวมีตั้งแต่ 0 ดาว ถึง 5 ดาว)


In [0]:
projects_stars = {
    "Client 1": {"Project 1": 5, "Project 2": 2.5, "Project 5": 4.5, "Project 6": 3.5, "Project 11": 1, "Project 12": 5, "Project 21": 0, "Project 22": 0, "Project 25": 2.5}, 
    "Client 2": {"Project 1": 3.5, "Project 2": 3.5, "Project 3": 5, "Project 4": 4.5, "Project 5": 0, "Project 6": 0, "Project 10": 2.5, "Project 11": 5, "Project 12": 4.5, "Project 20": 4.5, "Project 21": 1, "Project 22": 1.5},
    "Client 3": {"Project 3": 3.5, "Project 4": 4.5, "Project 5": 5, "Project 6": 2.5, "Project 10": 1.5, "Project 11": 1, "Project 12": 3, "Project 16": 4.5, "Project 17": 4.5, "Project 18": 4.5, "Project 20": 2, "Project 22": 2.5}, 
    "Client 4": {"Project 2": 4, "Project 3": 3.5, "Project 4": 5, "Project 5": 4.5, "Project 6": 2, "Project 7": 4, "Project 10": 5, "Project 20": 1, "Project 22": 1.5, "Project 25": 2.5}, 
    "Client 5": {"Project 1": 4.5, "Project 4": 4.5, "Project 5": 5, "Project 6": 4.5, "Project 7": 2, "Project 10": 4, "Project 11": 5, "Project 18": 1, "Project 20": 2.5, "Project 25": 4.5}, 
    "Client 6": {"Project 1": 5, "Project 3": 3.5, "Project 4": 5, "Project 5": 4.5, "Project 6": 2, "Project 7": 4, "Project 10": 5,  "Project 12": 3.5, "Project 20": 4.5, "Project 21": 1.5, "Project 22": 2.5}, 
    "Client 7": {"Project 2": 0, "Project 3": 3.5, "Project 4": 5, "Project 5": 4.5, "Project 6": 2, "Project 7": 4, "Project 10": 5, "Project 20": 1, "Project 22": 1.5, "Project 25": 5}, 
    "Client 8": {"Project 1": 4.5, "Project 2": 2.5, "Project 3": 0, "Project 4": 1.5, "Project 5": 0.5, "Project 6": 0.5, "Project 7": 3.5, "Project 10": 5, "Project 12": 3.5, "Project 20": 2.5, "Project 21": 1.5, "Project 22": 1.5, "Project 25": 3.5},
}

Recommendations จาก popularity

วิธีง่ายๆที่สามารถใช้ได้คือการดูภาพรวมว่าโปรเจคไหนได้ดาวเยอะที่สุด และทำการแนะนำโปรเจคที่ผู้ใช้งานยังไม่เคยให้ดาว
ขั้นตอนแรกเราจะทำการนับจำนวนดาวที่แต่ละผู้ใช้งานได้ให้ดาวไว้


In [0]:
max_stars = {"Project " + str(i):0 for i in range(1,30)}
for projects in projects_stars.keys():
    for project in projects_stars[projects]:
        max_stars[project] = projects_stars[projects][project]

In [0]:
import operator
sorted_x = sorted(max_stars.items(), key=operator.itemgetter(1), reverse=True)

In [0]:
def recommend_popular(user, star_list, results=5):
    recommendations = [(project, stars) for project, stars in star_list if project not in projects_stars[user]][:results]
    return recommendations

In [0]:
recommend_popular('Client 4', sorted_x)

Collaborative filtering 

ซึ่งเป็นการนำข้อมูลของผู้ใช้งานหลายๆคนมามองหาว่าใครมีความชอบคล้ายๆกับใคร และลองแนะนำผลงานอื่นๆที่ผู้ใช้งานน่าจะชอบ โดยดูจากคนที่มีความชอบคล้ายๆผู้ใช้งานคนนั้นๆชอบ

## Customer Segmentation

In [0]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans # Kmeans

# plotting package
import matplotlib.pyplot as plt
import seaborn as sns 

In [0]:
## Download Dataset
!wget https://ar686.s3-ap-southeast-1.amazonaws.com/CLV.csv

In [0]:
#Read the csv file
data_df = pd.read_csv('CLV.csv')

In [0]:
#Explore the dataset
data_df.head()#top 5 columns


In [0]:
# Visualization of the result

plt.scatter(x = data_df['INCOME'],
            y = data_df['SPEND'])

plt.xlabel('Income')
plt.ylabel('Spend')

plt.show()

In [0]:
SSE_list = [] # initilize SSE empty list

for k in range(1, 10):
    
    model = KMeans(n_clusters = k) # create model
    model.fit(data_df) # fit model to X data
    
    SSE = model.inertia_ # get the inertia  sum of squared error
    SSE_list.append(SSE) # keep result to list
    
    print('Current k is %d, sum distance = %.1f '%(k, SSE)) 
    # print the result to keep track performance

In [0]:
# This is a function to fit model and visualize the result basd on n_cluster = k


def plot_customer_segmentation(data_df, k):

    # create model
    model = KMeans(n_clusters = k)
    model.fit(data_df)
    y_pred = model.predict(data_df)

    # visualize result 
    plt.scatter(data_df.iloc[:,0],
                data_df.iloc[:,1], 
                c = y_pred
               )

    # add centroid point
    plt.scatter(model.cluster_centers_[:,0],
                model.cluster_centers_[:,1],
                s=100, # size of marker
                marker='s', # shape of marker
                c='red', # color of marker
                alpha=0.7, # transparency
               )  


    # decoration
    plt.xlabel(data_df.columns[0])
    plt.ylabel(data_df.columns[1])
    plt.title('Result of K-Mean Clustering of %d Cluster'%k)


    plt.show() # show the scatter plot



In [0]:
plot_customer_segmentation(data_df, 5)

### Exercise

ทดลองเปลี่ยนจำนวน k และลองพยายามตั้งชื่อกลุ่มของลูกค้าตามลักษณะข้อมูล

In [0]:
plot_customer_segmentation(data_df, 4)

#### ตัวอย่าง
* Cluster 1: Low income
* Cluster 2: Medium income with low spending    
* Cluster 3: Medium income with high spending 
* Cluster 4: High income   

## Prediction with Regression 1

In [0]:
from sklearn.linear_model import LinearRegression # Linear regression model
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
!wget https://ar686.s3-ap-southeast-1.amazonaws.com/TravelTime_HomeOffice.xlsx

In [0]:
data_df = pd.read_excel('TravelTime_HomeOffice.xlsx')
data_df.head()

In [0]:
# set X, y parameter
X = data_df[['Delay']] # X must be dataframe, not series : [[col_name]]
y = data_df['TravelTime']

In [0]:
# visualization of the data

plt.scatter(X,y)

plt.xlabel('Delay Departure Time from 5:30 AM')
plt.ylabel('Travel Time to Office (min)')

plt.show()

In [0]:
# create model
model = LinearRegression() 

In [0]:
# fit model with x and y data
# note: if use pandas, X must be dataframe as an input for this model that's why [[]]
model.fit(X,y) 

In [0]:
y_pred = model.predict(X)
y_pred

In [0]:
plt.scatter(X,y)
plt.plot(X,y_pred,'r') #[Added] plot the line
plt.text(40,40,'Y = %.4fX + %.4f' % (model.coef_, model.intercept_))

plt.xlabel('Delay Departure Time from 5:30 AM')
plt.ylabel('Travel Time to Office (min)')

plt.show()

## Prediction with Regression 2

In [0]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt


In [0]:
!wget https://ar686.s3-ap-southeast-1.amazonaws.com/Condo_ML.xlsm

In [0]:
df_train = pd.read_excel('Condo_ML.xlsm', sheet_name='Training')
df_train.head()

In [0]:
X_train = df_train[['YearBuilt', 'NFloor','DistParagon']]
y_train = df_train['SalePrice']

In [0]:
X_train

In [0]:
y_train

In [0]:
# Multivariate Linear Regression Model (ฺBasic สุดๆละ)
model = LinearRegression() # create model
model.fit(X_train,y_train) # fit with X and y

In [0]:
df_pred  = pd.read_excel('Condo_ML.xlsm', sheet_name='Prediction')
df_pred.head()

In [0]:
X_pred = df_pred[['YearBuilt', 'NFloor','DistParagon']]
print(X_pred.head())


In [0]:
y_pred = model.predict(X_pred)
print(y_pred)


## Face Detection (Image)

In [0]:
!pip install -q -U imutils git+https://github.com/the-house-of-black-and-white/hall-of-faces.git

In [0]:
%matplotlib inline

from matplotlib import pyplot as plt
from imutils import  url_to_image, opencv2matplotlib
from hof.face_detectors import YOLOv2FaceDetector

MIN_CONFIDENCE=0.5

def display(img, figsize=(15,15)):
  plt.figure(figsize=figsize)
  plt.axis('off')
  plt.imshow(opencv2matplotlib(img))
  plt.show()


In [0]:
selfie = url_to_image('https://pmcfootwearnews.files.wordpress.com/2017/02/oscars-ellen-degeneres-selfie.jpg?w=1024')
crowd = url_to_image('http://www.trbimg.com/img-540a7462/turbine/mc-central-at-whitehall-faces-in-the-crowd-20140905')
friends = url_to_image('https://www.sciencenewsforstudents.org/sites/default/files/2016/12/main/articles/860_main_beauty.png')
occluded = url_to_image('http://www.infotech.oulu.fi/Annual/2010/pics/mvg_06.jpg')

In [0]:
sample = crowd.copy()

In [0]:
yolo_face_detector = YOLOv2FaceDetector(min_confidence=MIN_CONFIDENCE)
yolo_face_detector.detect(sample, color=(15, 235, 250))
display(sample)

## Face Detection with Haar Cascades

In [0]:
from google.colab import files
file=files.upload()

In [0]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

Download Haar Cascades from https://github.com/opencv/opencv/tree/master/data/haarcascades

In [0]:
!wget https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml

In [0]:
!wget https://pmcfootwearnews.files.wordpress.com/2017/02/oscars-ellen-degeneres-selfie.jpg

In [0]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
img = cv2.imread('./oscars-ellen-degeneres-selfie.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
print(faces)

In [0]:
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    roigray = gray[y:y+h, x:x+w]
    roicolor = img[y:y+h, x:x+w]
plt.grid(None)   
plt.xticks([])
plt.yticks([])
imgplot = plt.imshow(img)

#### ทดลองกับภาพถ่ายของนักศึกษา และ Haar Cascade อื่นๆ

## Image Classification

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [0]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

In [0]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [0]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

In [0]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [0]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [0]:
sample_training_images, _ = next(train_data_gen)

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
plotImages(sample_training_images[:5])

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## MNIST Fashion

อ่านเพิ่มเติมที่ https://www.tensorflow.org/tutorials/keras/classification

This guide uses the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels), as seen here:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST is intended as a drop-in replacement for the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset—often used as the "Hello, World" of machine learning programs for computer vision. The MNIST dataset contains images of handwritten digits (0, 1, 2, etc.) in a format identical to that of the articles of clothing you'll use here.

This guide uses Fashion MNIST for variety, and because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

Here, 60,000 images are used to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from TensorFlow. Import and load the Fashion MNIST data directly from TensorFlow:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The *labels* are an array of integers, ranging from 0 to 9. These correspond to the *class* of clothing the image represents:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [0]:
train_images.shape

In [0]:
train_labels

In [0]:
test_images.shape

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(train_images, train_labels, epochs=10)

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [0]:
predictions = model.predict(test_images)

In [0]:
predictions[0]

In [0]:
np.argmax(predictions[0])

In [0]:
test_labels[0]

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()